In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pydub import AudioSegment
import os
from google.colab import files
import shutil

def merge_audio_files(input_folder, output_file):
    merged_audio = AudioSegment.silent(duration=0)

    for filename in os.listdir(input_folder):
        if filename.endswith(".mp3") or filename.endswith(".wav"):
            audio = AudioSegment.from_file(os.path.join(input_folder, filename))
            merged_audio += audio

    merged_audio.export(output_file, format="mp3")

def split_audio(input_file, output_folder, segment_length):
    audio = AudioSegment.from_file(input_file)
    total_length = len(audio)
    segment_length_ms = segment_length * 60 * 1000  # Convert segment length from minutes to milliseconds
    segment_start = 0
    segment_end = segment_length_ms

    # Create a temporary folder to store the segments
    temp_folder = os.path.join(output_folder, "temp")
    os.makedirs(temp_folder, exist_ok=True)

    while segment_end <= total_length:
        segment = audio[segment_start:segment_end]
        output_filename = f"segment_{int(segment_start/1000)}-{int(segment_end/1000)}.mp3"
        output_path = os.path.join(temp_folder, output_filename)
        segment.export(output_path, format="mp3")
        
        segment_start += segment_length_ms
        segment_end += segment_length_ms

    # Export the last segment if its duration is less than the segment length
    if segment_start < total_length:
        segment = audio[segment_start:]
        output_filename = f"segment_{int(segment_start/1000)}-{int(total_length/1000)}.mp3"
        output_path = os.path.join(temp_folder, output_filename)
        segment.export(output_path, format="mp3")

    # Zip the temporary folder containing the segments
    shutil.make_archive(output_folder, 'zip', temp_folder)

    # Generate a download link for the zip file
    files.download(output_folder + ".zip")

    # Remove the temporary folder
    shutil.rmtree(temp_folder)

# Example usage
input_folder = "/content/audio"
output_file = "/content/audio/merged_audio.mp3"
output_folder = "/content/result"
segment_length = 5  # Segment length in minutes

# Merge audio files
merge_audio_files(input_folder, output_file)

# Split the merged audio into 5-minute segments and download all segments at once
split_audio(output_file, output_folder, segment_length)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import librosa
import os
from sklearn.preprocessing import LabelEncoder

# Define the path to your audio folders
sleep_folder = "Audio\Sleep_audio\sleep"
engaging_folder = "Audio\good oration\engaging"
test_audio="Audio\\test audios\\test_audio_samples"

# Define the audio feature extraction parameters
sampling_rate = 44100
n_mfcc = 13
frame_length = 0.025
frame_hop = 0.01

# Initialize empty lists for features and labels
features = []
labels = []

# Function to extract features from audio files in a folder
def extract_features_from_folder(folder, class_label):
    for filename in os.listdir(folder):
        if filename.endswith(".mp3"):
            filepath = os.path.join(folder, filename)

            # Load the audio file
            audio, _ = librosa.load(filepath, sr=sampling_rate)

            # Extract MFCC features
            mfcc = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=n_mfcc,
                                        hop_length=int(frame_hop * sampling_rate),
                                        n_fft=int(frame_length * sampling_rate))

            # Flatten the MFCC features
            mfcc_flat = mfcc.flatten()

            # Add the features and labels to the lists
            features.append(mfcc_flat)
            labels.append(class_label)

# Extract features from the "sleep" folder
extract_features_from_folder(sleep_folder, "sleep")

# Extract features from the "engaging" folder
extract_features_from_folder(engaging_folder, "engaging")

#extract features from test audio folder
extract_features_from_folder(test_audio, "none")

# Convert the lists to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Preprocess the features if necessary (e.g., normalization)

# Encode the labels to numerical values
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Shuffle the data
random_indices = np.random.permutation(len(features))
features = features[random_indices]
encoded_labels = encoded_labels[random_indices]

# Save the preprocessed features and labels
np.save("Audio\\test audios/processed_features.npy", features)
np.save("Audio\\test audios/labels.npy", encoded_labels)

# Save the label encoder for later use
np.save("Audio\\test audios/label_encoder.npy", label_encoder.classes_)